# Trace Clustering

### Evaluating traces using a trace evaluator

Let $\mathcal{P}_g$ be the (possibly infinite) set of LTL predicates that can be generated from an LTL grammar $g$. A trace evaluator $\rho$ can be understood as a function that takes a predicate $p \in \mathcal{P}_g$ and a real-valued trace $t = \{s_1, \ldots, s_{\text{len}(t)}\}$ in a given trace space $\mathcal{T}$ and outputs an array of fitness values representing how well the predicate models the trace according to the function's criteria in a series of dimensions. More formally, if $m \in \mathbb{N}$ is the number of fitness dimensions (or alternatively, the number of metrics that are computed for each trace), this is a function with signature $\rho : \mathcal{T} \times \mathcal{P}_g \rightarrow \mathbb{R}^{m}$:

$$\rho (t, p) = (\rho_1(t, p), \ldots, \rho_{m}(t, p)),$$

where $\rho_{i}(t, p) \in \mathbb{R}$, $0 \leq i \leq m$, is the $i$-th fitness value of the trace $t$ with respect to the predicate $p$.

This allows us to convert a set of traces into a real vector representation that can be used to compare traces with each other.

One of the main trace evaluators that we will use is the `AutomatonTraceEvaluator`. This is based on the evaluation of a trace against a given predicate using an equivalent automaton representation (FSPA) of that predicate, while computing execution metrics to measure performance in different dimensions. Currently, the metrics that are computed are `progress` (how much of the automaton's specification is fulfilled by the trace), `waste` (how much of the trace is computed by the automaton before rejecting of accepting it), and `reward` (a measure of how many automaton steps were actually meaningful in advancing in the specification).

We will now run an example of how to use the `AutomatonTraceEvaluator` to evaluate traces against a predicate.

In [1]:
import predcompiler.compilation.io.readTraces
import kotlin.io.path.Path

// read the traces from the samples folder
val tracesData = readTraces(Path("../main/resources/samples/stealth/examples"))
tracesData.atomicPredicates

[bananaCollected01, banana02SafeFromCamera, bananaCollected03, bananaCollected02, bananaCollected05, bananaCollected04, bananaCollected07, bananaCollected06, bananaCollected09, bananaCollected08, playerCrouching, playerSafeFromCamera, playerSafeFromRobot, goalReached]

We will now evaluate the traces using the `AutomatonTraceEvaluator` and the predicate `SSEQ[bananaCollected01, bananaCollected02, bananaCollected03, bananaCollected04]`. This predicate corresponds to collecting bananas 1-4 in *strict* order. In the sample dataset, half of the traces can be modelled correctly by this predicate, while the other half cannot. In terms of the metrics computed by the evaluator, the traces that can be modelled correctly will have a `progress` of 1f (that is, the entire specification will be fulfilled in those cases), while the traces that cannot be modelled will have progress strictly lower than 1f. In reality, the traces that cannot be modelled will have a `progress` of 0f, as the predicate imposes a series of safety guarantees internally that instruct it to reject any run that visits the wrong banana before the right one (please note that in the current implementation, rejection implies a progress of 0f, but this can be tweaked depending on the goal at hand).

In [2]:
import predcompiler.compilation.evaluation.evaluators.traceset.evaluateTraceSet
import predcompiler.compilation.evaluation.evaluators.trace.AutomatonTraceEvaluator

// to create dataframes
%use dataframe

// evaluate the traces using an automaton trace evaluator and the predicate described above.
val predicate = "SSEQ[bananaCollected01, bananaCollected02, bananaCollected03, bananaCollected04]"
val evaluationResults = evaluateTraceSet(predicate, tracesData.traces, listOf(AutomatonTraceEvaluator()))
// convert the evaluation results to a dataframe for plotting
val metricRecords = evaluationResults.entries.map { Pair(it.key, it.value) }.toTypedArray()
var df = dataFrameOf(*metricRecords)
val category by columnOf(*Array(tracesData.traces.size) { it < tracesData.traces.size / 2 })
df

waste,reward,progress
0.879310,0.586207,1.000000
0.884058,0.594203,1.000000
0.909091,0.530303,1.000000
0.882353,0.514706,1.000000
0.924242,0.575758,1.000000
0.901639,0.704918,1.000000
0.910448,0.597015,1.000000
0.838710,0.612903,1.000000
0.861111,0.500000,1.000000
0.744186,0.674419,0.000000


We will now plot the evaluation results of the traces using the `kandy` library. In particular, we will be plotting the `reward` and `waste` metrics of the traces. In these examples, traces modeled by the predicate display better waste scores, but this doesn't necessarily have to be the case in general. A predicate may model a trace completely and still have a low waste score, as the trace may include a lot of irrelevant information that the predicate doesn't care about. This is usually the case when the predicate only refers to specific subsections of the behavior represented in the trace. Despite this, accounting for waste is often desirable when *searching* for predicates that model a set of traces, as it can help to identify predicates that explain the behavior of the traces in a more thorough way instead of focusing on specific pieces of information.

In this case, a trace spent entirely approaching the first banana would have a perfect waste score and a likely high reward metric because the automaton would never truly accept or reject the run and would continue to progress in the specification in a consistent manner. That trace's progress would be slow because it only covered a small portion of the predicate's goals, but the waste and reward metrics would be high. This is a fine example of how these metrics can be used to identify various types of behavior in traces and how they must be carefully weighed depending on the context of the evaluation. In general, a good predicate will have a mix of these characteristics.

In [3]:
import kotlin.reflect.KType

// plotting library
%use kandy

fun plotSamples(
    df: DataFrame<*>,
    category: DataColumn<Boolean>,
    trueText: String,
    falseText: String,
    plotTitle: String
) {
    // check that dataframe contains the necessary columns
    assert(df.containsColumn("waste") && df.containsColumn("reward") && df.containsColumn("progress"), 
        { "The dataframe must contain the columns 'waste', 'reward', and 'progress'." })
    // check that dataframe has as many rows as the categories column
    assert(df.rowsCount() == category.size, 
        { "The dataframe must have the same number of rows as the categories column." })
    // check that the waste, reward and progress columns are numeric
    assert(
        df["waste"].type.classifier == Double::class &&
        df["reward"].type.classifier == Double::class &&
        df["progress"].type.classifier == Double::class, 
        { "The 'waste', 'reward', and 'progress' columns must be numeric." })

    val modelled by columnOf(*Array(df.rowsCount()) { (df[it].get("progress") as Double) >= 1.0 })

    val newCat = category.rename("category")
    df
        .add(newCat)
        .add(modelled)
        .plot {
            points {
                x("waste") { scale = continuous(0.0..1.0) }
                y("reward") { scale = continuous(0.0..1.0) }
                size("progress") { scale = continuous(2.0..6.0) }
                color("category") {
                    scale = categorical(
                        true to Color.YELLOW, false to Color.BLUE
                    )
                    legend {
                        name = "Category"
                        breaksLabeled(true to trueText, false to falseText)
                    }
                }
                symbol("modelled") {
                    scale = categorical(true to Symbol.BULLET, false to Symbol.CROSS)
                }
            }
            layout.title = plotTitle
        }
        .save("${plotTitle}.png")
} // plotSamples

In [4]:
plotSamples(df, category, "Modelled", "Not modelled", "Sample Split by Modelled Predicate")

### Clustering traces based on their evaluation results

Now that we've taken a look at how evaluators work and what sorts of results they may output for a set of traces, we can move on to clustering the traces by searching for predicates that split the samples into well-defined behavior groups. 

In [5]:
// data analysis library
%use smile

val wasteEntries = evaluationResults["waste"]!!
val rewardEntries = evaluationResults["reward"]!!
val progressEntries = evaluationResults["progress"]!!

val clusteringData = Array(tracesData.traces.size) { 
    doubleArrayOf(wasteEntries[it], rewardEntries[it], progressEntries[it])
}

val clusteringResult = kmeans(clusteringData, 2)
val cluster by columnOf(*Array(clusteringData.size) { clusteringResult.predict(clusteringData[it]) >= 1 })
clusteringResult

Cluster distortion: 0.07655
Cluster size of 18 data points:
Cluster    1      9 (50.0%)
Cluster    2      9 (50.0%)


In [6]:
plotSamples(df, cluster, "Cluster A", "Cluster B", "Sample Split by Predicted Cluster")